# 目标检测
读取视频，并检测目标，引用yolov5预训练模型。

参考：<br>
[Real-Time-Object-Detection](https://github.com/akash-agni/Real-Time-Object-Detection)<br>
[实战：使用 PyTorch 和 OpenCV 实现实时目标检测系统](https://mp.weixin.qq.com/s?__biz=MzU0NjgzMDIxMQ==&mid=2247592410&idx=1&sn=fb987ba96efae6b7f977aa53a21d784f&chksm=fb548b36cc230220c349316e0bac922476f8f07d99c5b344602e168c65e06ac687da6b349c52&mpshare=1&scene=1&srcid=0512wwvosev6RrXyuHyuj1F7&sharer_sharetime=1652321842848&sharer_shareid=46709a3b09e682ea1b4eea0fb295a91c#rd)


In [1]:
import cv2

In [3]:
import torch
from torch import hub
import time
import numpy as np

opencv读取视频并截图<br>
- 读取本地视频：输入路径  `cv2.VideoCapture('./Yoku.mp4')`
- 读取本地摄像头：值为0  `cv2.VideoCapture(0)`

In [26]:
class ObjectDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using Opencv2.
    """

    def __init__(self, url, out_file="Labeled_Video.avi"):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        #play = pafy.new(self._URL).streams[-1]
        #assert play is not None
        #return cv2.VideoCapture(play.url)
        stream = cv2.VideoCapture('./Yoku.mp4')
        #stream = cv2.VideoCapture(0)
        return stream
        

    def load_model(self):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        player = self.get_video_from_url()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        while True:
            start_time = time.time()
            ret, frame = player.read()
            assert ret
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time.time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)


In [27]:
# Create a new object and execute.
a = ObjectDetection('test')
a()

Using cache found in C:\Users\ingse/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-5-15 torch 1.11.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Frames Per Second : 2.1186440677966103
Frames Per Second : 2.232142857142857
Frames Per Second : 1.6260162601626016
Frames Per Second : 2.0080321285140563
Frames Per Second : 2.202643171806167
Frames Per Second : 2.0876826722338206
Frames Per Second : 2.1645021645021645
Frames Per Second : 1.976284584980237
Frames Per Second : 1.8115942028985506
Frames Per Second : 1.8796992481203008
Frames Per Second : 2.0161290322580645
Frames Per Second : 2.202643171806167
Frames Per Second : 1.2836970474967908
Frames Per Second : 2.070393374741201
Frames Per Second : 1.8018018018018016
Frames Per Second : 1.7094017094017095
Frames Per Second : 2.232142857142857
Frames Per Second : 2.061855670103093
Frames Per Second : 1.7543859649122808
Frames Per Second : 1.5082956259426847
Frames Per Second : 1.7574692442882252
Frames Per Second : 2.2831050228310503
Frames Per Second : 2.232142857142857
Frames Per Second : 1.7391304347826089
Frames Per Second : 2.0161290322580645
Frames Per Second : 1.21065375302

Frames Per Second : 1.9011406844106462
Frames Per Second : 1.953125
Frames Per Second : 2.3696682464454977
Frames Per Second : 2.070393374741201
Frames Per Second : 1.8903591682419658
Frames Per Second : 1.996007984031936
Frames Per Second : 1.8796992481203008
Frames Per Second : 1.9047619047619047
Frames Per Second : 2.053388090349076
Frames Per Second : 1.9342359767891681
Frames Per Second : 1.9120458891013383
Frames Per Second : 1.8796992481203008
Frames Per Second : 2.3980815347721824
Frames Per Second : 2.2271714922048997
Frames Per Second : 2.0242914979757085
Frames Per Second : 1.8796992481203008
Frames Per Second : 1.4925373134328357
Frames Per Second : 1.9920318725099602
Frames Per Second : 1.773049645390071
Frames Per Second : 1.6778523489932886
Frames Per Second : 1.1695906432748537
Frames Per Second : 1.8181818181818181
Frames Per Second : 2.05761316872428
Frames Per Second : 1.2903225806451613
Frames Per Second : 0.8920606601248885
Frames Per Second : 1.0638297872340425
Fr

Frames Per Second : 2.092050209205021
Frames Per Second : 2.247191011235955
Frames Per Second : 2.4038461538461537
Frames Per Second : 2.202643171806167
Frames Per Second : 2.314814814814815
Frames Per Second : 2.3041474654377883
Frames Per Second : 2.0080321285140563
Frames Per Second : 1.8148820326678765
Frames Per Second : 2.100840336134454
Frames Per Second : 1.976284584980237
Frames Per Second : 2.178649237472767
Frames Per Second : 2.232142857142857
Frames Per Second : 2.127659574468085
Frames Per Second : 2.314814814814815
Frames Per Second : 2.202643171806167
Frames Per Second : 2.044989775051125
Frames Per Second : 2.2988505747126435
Frames Per Second : 2.0491803278688523
Frames Per Second : 1.7574692442882252
Frames Per Second : 2.380952380952381
Frames Per Second : 2.150537634408602
Frames Per Second : 1.8832391713747645
Frames Per Second : 2.277904328018223
Frames Per Second : 2.159827213822894
Frames Per Second : 2.247191011235955
Frames Per Second : 2.202643171806167
Fram

Frames Per Second : 2.202643171806167
Frames Per Second : 2.0964360587002098
Frames Per Second : 1.7035775127768313
Frames Per Second : 1.7761989342806397
Frames Per Second : 2.0
Frames Per Second : 1.5822784810126582
Frames Per Second : 1.4662756598240467
Frames Per Second : 2.074688796680498
Frames Per Second : 1.9723865877712032
Frames Per Second : 1.9801980198019802
Frames Per Second : 2.032520325203252
Frames Per Second : 1.953125
Frames Per Second : 2.053388090349076
Frames Per Second : 2.237136465324385
Frames Per Second : 2.314814814814815
Frames Per Second : 2.202643171806167
Frames Per Second : 1.9342359767891681
Frames Per Second : 2.05761316872428
Frames Per Second : 2.232142857142857
Frames Per Second : 2.127659574468085
Frames Per Second : 1.9607843137254901
Frames Per Second : 2.293577981651376
Frames Per Second : 2.159827213822894
Frames Per Second : 2.202643171806167
Frames Per Second : 2.202643171806167
Frames Per Second : 2.314814814814815
Frames Per Second : 2.12765

Frames Per Second : 1.9047619047619047
Frames Per Second : 1.996007984031936
Frames Per Second : 1.6528925619834711
Frames Per Second : 1.6722408026755853
Frames Per Second : 1.3089005235602094
Frames Per Second : 0.8960573476702508
Frames Per Second : 1.272264631043257
Frames Per Second : 1.968503937007874
Frames Per Second : 2.202643171806167
Frames Per Second : 2.1141649048625792
Frames Per Second : 1.7605633802816902
Frames Per Second : 1.5847860538827259
Frames Per Second : 1.5822784810126582
Frames Per Second : 2.05761316872428
Frames Per Second : 1.8796992481203008
Frames Per Second : 1.996007984031936
Frames Per Second : 2.0


AssertionError: 